In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
#minha funcao para mostrar as imagens
def imshow(im,big=False):
    if big:
        plt.figure(figsize=(20,20))
    else:
        plt.figure()
    if len(im.shape)<3:
        plt.imshow(im, cmap='gray')#As cores ficam zoadas no Jupyter se deixar sem esse parâmetro
    else:
        plt.imshow(im) 
    plt.axis('off')

In [ ]:
def f2i(im):#float to int
    return np.round(im * 255.0).astype('uint8')

### 1. Abrindo Imagem

60.bmp<br>
82.bmp<br>
114.bmp<br>
150.bmp<br>
205.bmp

In [ ]:
im = cv2.imread("205.bmp")
imshow(im)

### 2. Convertendo para escala de cinza

In [ ]:
im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
imshow(im_gray)

### Normalização local

Reduzir problemas de iluminação na imagem

In [ ]:
tamanho_janela = 51
kernel = np.ones((tamanho_janela, tamanho_janela),np.uint8)
minimo = cv2.erode(im_gray, kernel)
maximo = cv2.dilate(im_gray, kernel)

im_norm_local = (im_gray - minimo) / (maximo - minimo)

imshow(im_norm_local)

### Filtro da Mediana

Homogeneizar as texturas

In [ ]:
tamanho_janela = 9
im_borrada = cv2.medianBlur(f2i(im_norm_local), tamanho_janela)
imshow(im_borrada)

### Dilata e cria uma mascara

In [ ]:
ret,mask = cv2.threshold(im_borrada,200, 255, cv2.THRESH_BINARY)
mask = cv2.dilate(mask,np.ones((3,3),'uint8'),iterations=2)
imshow(mask)

In [ ]:
im_sem_fundo = im_norm_local.copy()
im_sem_fundo[mask==0] = 0
imshow(im_sem_fundo)

### Limiarização global

In [ ]:
ret,im_limiarizacao_global = cv2.threshold(im_sem_fundo,0.8, 1.0, cv2.THRESH_BINARY)
imshow(im_limiarizacao_global)

### Morfologias

In [ ]:
#im_erode = cv2.erode(im_limiarizacao_global,kernel,iterations = 1)
#im_close = cv2.morphologyEx(im_limiarizacao_global, cv2.MORPH_CLOSE, kernel)
#imshow(im_close,True)

Não achei nenhuma morfologia boa ainda, entao não faço essa etapa

In [ ]:
im_morf = im_limiarizacao_global

### Rotulagem

In [ ]:
ret, labels = cv2.connectedComponents(im_limiarizacao_global.astype('uint8'))
ret

In [ ]:
def pinta_labels(labels):
    # Map component labels to hue val
    label_hue = np.uint8(1500*labels/np.max(labels))
    blank_ch = 255*np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

    # cvt to BGR for display
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)

    # set bg label to black
    labeled_img[label_hue==0] = 0
    
    return labeled_img

In [ ]:
im_labels = pinta_labels(labels)
imshow(im_labels)

# 1 Tentativa - Baseado nas Areas
A ideia é dividir a area de todos os blobs pelo tamanho medio de um arroz

### Areas

In [ ]:
areas = np.zeros(ret).astype(int)
for i in range(ret):
    areas[i] = np.sum(labels==i)
areas = areas[1:]#removendo os zeros 
areas = areas[areas>20]# removendo as muito pequenas
areas.sort()
areas

### Quase Moda
tentando achar o tamanho medio de um arroz por uma aproximação da moda

In [ ]:
import numpy as np
from scipy import stats
mode = stats.mode(np.round(areas/10)).mode[0]*10
mode

In [ ]:
print('Estimativa')
np.round(areas/mode).sum()

### Para salvar o arquivo localmente

In [ ]:
cv2.imwrite("./debug.png", im_borrada)